# Export workflow definitions

We need to compose workflows and place the function prototypes into their proper place.

### IMports


In [1]:
from infer_subc.organelles_config.helper import write_workflow_json


In [2]:
from  infer_subc.constants import *

from typing import Dict, List


-----------------
## WORKFLOWS



_____________

##  MASKS, NUCLEI, CELLMASK, CYTOPLASM

In [43]:
def make_infer_masks_fromaggr_MCZ():
    """
    crete .json version workflow for gettnig masks (using cellmask from MCZ)
    """
    """
    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    step_n = 1
    ###################
    # EXTRACT
    ###################   
    raw_input_step = 0


    step_name.append(f"{step_n}")
    function_name.append("infer_nuclei_fromlabel")
    category.append("extraction")
    parameter_values.append( dict(
                                nuc_ch = NUC_CH,
                                median_sz = 3,
                                gauss_sig = 5.,
                                thresh_factor = 0.8,
                                thresh_min = 0.1,
                                thresh_max = 1.0,
                                max_hole_w = 35,
                                small_obj_w = 15)
                                )
                                
    parent.append(raw_input_step)
    annotation.append(f"get  nuclei segmentation: {step_n}")    
    nuc_step = step_n
    step_n += 1                     

    step_name.append(f"{step_n}")
    function_name.append("raw_cellmask_fromaggr")
    category.append("extraction")
    parameter_values.append(dict(scale_min_max=True))
    parent.append(raw_input_step)
    annotation.append(f" this creates an aggregate signal for the cellmask" )
    cellmask_input_step = step_n

    step_n += 1                     
    ### PRE PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 4, gauss_sig=4.5 ))
    parent.append(cellmask_input_step)
    annotation.append(f"cellmask segmentation: {step_n}")
    struct_img_step = step_n
    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("non_linear_cellmask_transform")
    category.append("preprocessing")
    parameter_values.append(None)
    annotation.append(f"cellmask segmentation: {step_n}")
    parent.append(step_n-1)

    step_n += 1

    ### CORE
    step_name.append(f"{step_n}")
    function_name.append("masked_object_thresh")
    category.append("core")
    parameter_values.append(dict( th_method="ave_tri_med",
                                                            cutoff_size = 375,
                                                            th_adjust = 0.94))
    parent.append(step_n-1)
    annotation.append(f"cellmask segmentation: {step_n}")

    step_n += 1

    ### POST_PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 55, 
                                  min_size = 42,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"cellmask segmentation: {step_n}")
    
    step_n += 1

    ### POST- POST_PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("choose_max_label_cellmask_union_nucleus")
    category.append("postpostprocessing")
    parameter_values.append(dict(interior_labels=True))
    annotation.append(f"cellmask segmentation: {step_n}")
    parent.append([struct_img_step, step_n-1,nuc_step])
    cellmask_step = step_n

    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("infer_cytoplasm")
    category.append("postpostprocessing")
    parameter_values.append(dict(erode_nuclei=True))
    parent.append([nuc_step, cellmask_step])
    annotation.append(f"infer cytoplasm: {step_n}")
    cyto_step = step_n

    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("stack_masks")
    category.append("export")
    parameter_values.append(None)
    parent.append([nuc_step, cellmask_step, cyto_step])
    annotation.append(f"export  canonical masks (nuc,cellmask, cyto) as np.uint16{step_n}")
    output_step = step_n


    # ###################
    # # EXPORT
    # ###################
    # step_name.append("18")
    # function_name.append("export_PLACE_HOLDER")
    # category.append("export")
    # parameter_values.append(None)
    # parent.append([5, 17])


    ##########################
    out_dict = dict()
    for i,stepn in enumerate(step_name):

        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i])
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [44]:

infer_masks_MCZ_dict = make_infer_masks_fromaggr_MCZ()


write_workflow_json("conf_0.1.masks_MCZ", infer_masks_MCZ_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc/infer_subc/organelles_config/conf_0.1.masks_MCZ.json')

In [45]:
from infer_subc.constants import NUC_CH

def make_infer_mask_fromaggr_step_by_step_dict():
    """
    crete .json version of infer_cellmask_fromaggr
    """
    """
    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    ###################
    # EXTRACT
    ###################   
    raw_input_step = 0
    step_n = 1

    step_name.append(f"{step_n}")
    function_name.append("infer_nuclei_fromlabel")
    category.append("extraction")
    parameter_values.append( dict(
                                nuc_ch = NUC_CH,
                                median_sz = 3,
                                gauss_sig = 4.5,
                                thresh_factor = 0.85,
                                thresh_min = 0.1,
                                thresh_max = 1.0,
                                max_hole_w = 25,
                                small_obj_w = 15)
                                )
    parent.append(raw_input_step)
    annotation.append(f"get  nuclei segmentation: {step_n}")    
    nuc_step = step_n

    step_n += 1                     

    step_name.append(f"{step_n}")
    function_name.append("make_aggregate")
    category.append("extraction")
    parameter_values.append(dict(w0=0,
                                    w1=6,
                                    w2=0,
                                    w3=2,
                                    w4=0,
                                    w5=1,
                                    w6=0,
                                    w7=0,
                                    w8=0,
                                    w9=0,
                                    scale_min_max = True) 
                )
    parent.append(raw_input_step)
    annotation.append(f" this creates an aggregate signal for the cellmask" )
    cellmask_input_step = step_n

    step_n += 1                   

    ### PRE PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 9, gauss_sig=4.5 ))
    parent.append(cellmask_input_step)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    struct_img_step = step_n
    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("non_linear_cellmask_transform")
    category.append("preprocessing")
    parameter_values.append(None)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    parent.append(step_n-1)

    step_n += 1

    ### CORE
    step_name.append(f"{step_n}")
    function_name.append("masked_object_thresh")
    category.append("core")
    parameter_values.append(dict( th_method="ave_tri_med",
                                                            cutoff_size = 200,
                                                            th_adjust = 0.75))
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")

    step_n += 1

    ### POST_PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 45, 
                                  min_size = 30,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    
    step_n += 1

    ### POST- POST_PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("choose_max_label_cellmask_union_nucleus")
    category.append("postpostprocessing")
    parameter_values.append(dict(interior_labels=True))
    annotation.append(f"basic cellmask segmentation: {step_n}")
    parent.append([struct_img_step, step_n-1,nuc_step])
    cellmask_step = step_n

    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("infer_cytoplasm")
    category.append("postpostprocessing")
    parameter_values.append(dict(erode_nuclei=True))
    parent.append([nuc_step, cellmask_step])
    annotation.append(f"infer cytoplasm: {step_n}")
    cyto_step = step_n


    cyto_step = step_n

    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("stack_masks")
    category.append("export")
    parameter_values.append(None)
    parent.append([nuc_step, cellmask_step, cyto_step])
    annotation.append(f"export  canonical masks (nuc,cell, cyto) {step_n}")
    output_step = step_n


    # ###################
    # # EXPORT
    # ###################
    # step_name.append("18")
    # function_name.append("export_PLACE_HOLDER")
    # category.append("export")
    # parameter_values.append(None)
    # parent.append([5, 17])


    ##########################
    out_dict = dict()
    for i,stepn in enumerate(step_name):

        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [46]:

infer_cellmask_fromaggr_stepbystep_dict = make_infer_mask_fromaggr_step_by_step_dict()

write_workflow_json("conf_0.1.masks", infer_cellmask_fromaggr_stepbystep_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc/infer_subc/organelles_config/conf_0.1.masks.json')

-------------------------------
## LYSOSOME

In [31]:
from infer_subc.constants import LYSO_CH
def make_infer_lyso_step_by_step_from_raw_dict():
    """
    Procedure to infer lysosome from linearly unmixed input from raw

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    ###################
    # EXTRACT
    ###################   
    step_n = 1
    raw_input_step = 0

    step_name.append(f"{step_n}")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = LYSO_CH) )
    parent.append(raw_input_step)
    annotation.append(f"basic lyso segmentation: {step_n}")
    step_n += 1

    ###################
    # PRE_PROCESSING
    ###################                         
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 4, gauss_sig=1.4 ))
    parent.append(step_n-1)
    annotation.append(f"basic lyso segmentation: {step_n}")
    img_step = step_n
    step_n += 1

    ###################
    # CORE_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("spot_filter_3")
    category.append("core")
    parameter_values.append(dict( dot_scale_1 = 5,
                                dot_cut_1 = 0.09,
                                dot_scale_2 = 2.5,
                                dot_cut_2 = 0.07,
                                dot_scale_3 = 1,
                                dot_cut_3 = 0.01))
    parent.append(step_n-1)
    annotation.append(f"basic lyso - spot filter: {step_n}")
    spot_step = step_n
    step_n += 1


    step_name.append(f"{step_n}")
    function_name.append("filament_filter")
    category.append("core")
    parameter_values.append(dict(  filament_scale = 1.,
                                                        filament_cut = 0.15))
    parent.append(img_step)
    annotation.append(f"basic lyso - filament filter: {step_n}")
    fil_step = step_n
    step_n += 1


    step_name.append(f"{step_n}")
    function_name.append("logical_or")
    category.append("core")
    parameter_values.append( None )
    parent.append([spot_step,fil_step])
    annotation.append(f"basic lyso - combine spot+filament: {step_n}")
    step_n += 1


    ###################
    # POST_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 40, 
                                  min_size = 4,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic lyso - fill/filter: {step_n}")
    step_n += 1


    step_name.append(f"{step_n}")
    function_name.append("label_uint16")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append(step_n-1)
    annotation.append(f"label to uint16 for export {step_n}")
    output_step = step_n


    
    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        



    return out_dict

In [32]:

infer_lyso_stepbystep_from_raw_dict = make_infer_lyso_step_by_step_from_raw_dict()

write_workflow_json("conf_0.2.lyso", infer_lyso_stepbystep_from_raw_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc/infer_subc/organelles_config/conf_0.2.lyso.json')

-------------------------------
## MITO

In [33]:
from infer_subc.constants import MITO_CH 

def make_infer_mito_step_by_step_from_raw_dict():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    ###################
    # EXTRACT
    ###################   
    step_n = 1
    raw_input_step = 0

    step_name.append(f"{step_n}")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = MITO_CH) )
    parent.append(raw_input_step)
    annotation.append(f"basic mito segmentation: {step_n}")
    step_n += 1

    ###################
    # PRE_PROCESSING
    ###################                         
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 3, gauss_sig=2.6 ))
    parent.append(step_n-1)
    annotation.append(f"basic mito segmentation: {step_n}")
    step_n += 1


    ###################
    # CORE_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("vesselness_slice_by_slice")
    category.append("core")
    parameter_values.append( dict( sigma=1.5, cutoff=0.05))
    parent.append(step_n-1)
    annotation.append(f"basic mito - vesselness filter: {step_n}")
    spot_step = step_n
    step_n += 1



    ###################
    # POST_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 25, 
                                  min_size = 11,
                                 method="3D" ))
    parent.append(step_n-1)
    annotation.append(f"basic mito - fill/filter: {step_n}")
    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("label_uint16")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append(step_n-1)
    annotation.append(f"label to uint16 for export {step_n}")
    output_step = step_n

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    
    return out_dict



In [34]:

infer_mito_stepbystep_from_raw_dict = make_infer_mito_step_by_step_from_raw_dict()

write_workflow_json("conf_0.3.mito", infer_mito_stepbystep_from_raw_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc/infer_subc/organelles_config/conf_0.3.mito.json')

-------------------------------
## GOLGI

In [35]:
from infer_subc.constants import GOLGI_CH
def make_infer_golgi_step_by_step_from_raw_dict():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    ###################
    # EXTRACT
    ###################   
    step_n = 1
    raw_input_step = 0

    step_name.append(f"{step_n}")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = GOLGI_CH) )
    parent.append(raw_input_step)
    annotation.append(f"basic golgi segmentation: {step_n}")
    step_n += 1

    ###################
    # PRE_PROCESSING
    ###################                         
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 4, gauss_sig=1.4 ))
    parent.append(step_n-1)
    annotation.append(f"basic golgi segmentation: {step_n}")
    img_step = step_n
    step_n += 1


    ###################
    # CORE_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("masked_object_thresh")
    category.append("core")
    parameter_values.append(dict(th_method= "triangle", 
                                                            cutoff_size=1200,
                                                            th_adjust = 0.5) )
    parent.append(img_step)
    annotation.append(f"basic golgi - mo: {step_n}")
    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("topology_preserving_thinning")
    category.append("core")
    parameter_values.append(dict(    
                                                min_thickness = 1.6,
                                                thin = 1) )
    parent.append(step_n-1)
    annotation.append(f"basic golgi - thinning filter: {step_n}")
    fil_step = step_n
    step_n += 1
    
    step_name.append(f"{step_n}")
    function_name.append("spot_filter_3")
    category.append("core")
    parameter_values.append(dict( dot_scale_1 = 1.6,
                                dot_cut_1 = 0.02,
                                dot_scale_2 = 0,
                                dot_cut_2 = 0.1,
                                dot_scale_3 = 0,
                                dot_cut_3 = 0.1))
    parent.append(img_step)
    annotation.append(f"basic golgi - spot filter: {step_n}")
    spot_step = step_n
    step_n += 1


    step_name.append(f"{step_n}")
    function_name.append("logical_or")
    category.append("core")
    parameter_values.append( None )
    parent.append([spot_step,fil_step])
    annotation.append(f"basic golgi - combine spot+thinned: {step_n}")

    step_n += 1



    ###################
    # POST_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 25, 
                                  min_size = 11,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic mito - fill/filter: {step_n}")
    step_n += 1

    
    step_name.append(f"{step_n}")
    function_name.append("label_uint16")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append(step_n-1)
    annotation.append(f"label to uint16 for export {step_n}")
    output_step = step_n
    
    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    
    return out_dict



In [36]:

infer_golgi_stepbystep_from_raw_dict = make_infer_golgi_step_by_step_from_raw_dict()

write_workflow_json("conf_0.4.golgi", infer_golgi_stepbystep_from_raw_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc/infer_subc/organelles_config/conf_0.4.golgi.json')

-------------------------------
## PEROXISOME

In [37]:
from infer_subc.constants import PEROX_CH


def make_infer_perox_step_by_step_from_raw_dict():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    ###################
    # EXTRACT
    ###################   
    step_n = 1
    raw_input_step = 0

    step_name.append(f"{step_n}")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = PEROX_CH) )
    parent.append(raw_input_step)
    annotation.append(f"basic perox segmentation: {step_n}")
    step_n += 1

    ###################
    # PRE_PROCESSING
    ###################                         
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 1, gauss_sig=3.0 ))
    parent.append(step_n-1)
    annotation.append(f"basic perox segmentation: {step_n}")
    step_n += 1


    ###################
    # CORE_PROCESSING
    ###################

    step_name.append(f"{step_n}")
    function_name.append("spot_filter_3")
    category.append("core")
    parameter_values.append(dict( dot_scale_1 = 1.0,
                                dot_cut_1 = 0.01,
                                dot_scale_2 = 0,
                                dot_cut_2 = 0.1,
                                dot_scale_3 = 0,
                                dot_cut_3 = 0.1))
    parent.append(step_n-1)
    annotation.append(f"basic perox - spot filter (1 scale): {step_n}")
    step_n += 1


    ###################
    # POST_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 0, 
                                  min_size = 2,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic perox - fill/filter: {step_n}")
    step_n += 1

    
    step_name.append(f"{step_n}")
    function_name.append("label_uint16")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append(step_n-1)
    annotation.append(f"label to uint16 for export {step_n}")
    output_step = step_n
    
    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    
    return out_dict




In [38]:

infer_perox_stepbystep_from_raw_dict = make_infer_perox_step_by_step_from_raw_dict()

write_workflow_json("conf_0.5.perox", infer_perox_stepbystep_from_raw_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc/infer_subc/organelles_config/conf_0.5.perox.json')

-------------------------------
## ER


In [5]:
from infer_subc.constants import ER_CH
def make_infer_er_step_by_step_from_raw_dict():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw

    """
   
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    ###################
    # EXTRACT
    ###################   
    step_n = 1
    raw_input_step = 0

    step_name.append(f"{step_n}")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = ER_CH) )
    parent.append(raw_input_step)
    annotation.append(f"basic ER segmentation: {step_n}")
    step_n += 1

    ###################
    # PRE_PROCESSING
    ###################                         
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 1, gauss_sig=3.0 ))
    parent.append(step_n-1)
    annotation.append(f"basic ER segmentation: {step_n}")
    step_n += 1


    ###################
    # CORE_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("filament_filter")
    category.append("core")
    parameter_values.append(dict(  filament_scale = 1.,
                                                        filament_cut = 0.015))
    parent.append(step_n-1)
    annotation.append(f"basic ER - filament filter: {step_n}")
    fil_step = step_n
    step_n += 1


    ###################
    # POST_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 0, 
                                  min_size = 2,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic ER - fill/filter: {step_n}")
    step_n += 1

    
    step_name.append(f"{step_n}")
    function_name.append("label_bool_as_uint16")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append(step_n-1)
    annotation.append(f"label to uint16 for export {step_n}")
    output_step = step_n
    
    

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    
    return out_dict




In [6]:

infer_er_stepbystep_from_raw_dict = make_infer_er_step_by_step_from_raw_dict()

write_workflow_json("conf_0.6.ER", infer_er_stepbystep_from_raw_dict)

PosixPath('/Users/ergonyc/Projects/Imaging/infer-subc/infer_subc/organelles_config/conf_0.6.ER.json')

-------------------------------
## LD


In [41]:
from infer_subc.constants import LD_CH

def make_infer_LD_step_by_step_from_raw_dict():
    """
    Procedure to infer lipid from linearly unmixed input.

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels

    soma_mask: np.ndarray
        mask

    Returns
    -------------
    LD_object
        mask defined extent of NU

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    step_n = 1
    ###################
    # EXTRACT
    ###################   
    raw_input_step = 0


    step_name.append(f"{step_n}")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = LD_CH) )
    parent.append(step_n-1)
    annotation.append(f"basic LD segmentation: {step_n}")
    step_n += 1

    ###################
    # PRE_PROCESSING
    ###################                         
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 1, gauss_sig=2.4 ))
    parent.append(step_n-1)
    annotation.append(f"basic LD segmentation: {step_n}")
    step_n += 1

    ###################
    # CORE_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("apply_threshold")
    category.append("core")
    parameter_values.append(dict(method = "otsu",
                                                        thresh_factor = 0.7, 
                                                            thresh_min = .33,
                                                            thresh_max = 1.))
    parent.append(step_n-1)
    annotation.append(f"basic LD segmentation: {step_n}")
    step_n += 1

    ###################
    # POST_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 2, 
                                  min_size = 4,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic LD segmentation: {step_n}")
    step_n += 1


    step_name.append(f"{step_n}")
    function_name.append("label_uint16")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append(step_n-1)
    annotation.append(f"label to uint16 for export {step_n}")
    output_step = step_n
    
    
    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [42]:

infer_LD_stepbystep_from_raw_dict = make_infer_LD_step_by_step_from_raw_dict()

write_workflow_json("conf_0.7.LD", infer_LD_stepbystep_from_raw_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc/infer_subc/organelles_config/conf_0.7.LD.json')

In [26]:
from infer_subc.constants import NUC_CH

def make_infer_mask_fromaggr_step_by_step_dict():
    """
    crete .json version of infer_cellmask_fromaggr
    """
    """
    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    ###################
    # EXTRACT
    ###################   
    raw_input_step = 0
    step_n = 1

    step_name.append(f"{step_n}")
    function_name.append("infer_nuclei_fromlabel")
    category.append("extraction")
    parameter_values.append( dict(
                                nuc_ch = NUC_CH,
                                median_sz = 4,
                                gauss_sig = 1.34,
                                thresh_factor = 0.9,
                                thresh_min = 0.1,
                                thresh_max = 1.0,
                                max_hole_w = 25,
                                small_obj_w = 15)
                                )
    parent.append(raw_input_step)
    annotation.append(f"get  nuclei segmentation: {step_n}")    
    nuc_step = step_n

    step_n += 1                     

    step_name.append(f"{step_n}")
    function_name.append("make_aggregate")
    category.append("extraction")
    parameter_values.append(dict(w0=0,
                                    w1=6,
                                    w2=0,
                                    w3=2,
                                    w4=0,
                                    w5=1,
                                    w6=0,
                                    w7=0,
                                    w8=0,
                                    w9=0,
                                    scale_min_max = True) 
                )
    parent.append(raw_input_step)
    annotation.append(f" this creates an aggregate signal for the cellmask" )
    cellmask_input_step = step_n

    step_n += 1                   

    ### PRE PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 4, gauss_sig=1.4 ))
    parent.append(cellmask_input_step)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    struct_img_step = step_n
    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("non_linear_cellmask_transform")
    category.append("preprocessing")
    parameter_values.append(None)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    parent.append(step_n-1)

    step_n += 1

    ### CORE
    step_name.append(f"{step_n}")
    function_name.append("masked_object_thresh")
    category.append("core")
    parameter_values.append(dict( th_method="ave_tri_med",
                                                            cutoff_size = 100,
                                                            th_adjust = 0.5))
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")

    step_n += 1

    ### POST_PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 25, 
                                  min_size = 15,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    
    step_n += 1

    ### POST- POST_PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("choose_max_label_cellmask_union_nucleus")
    category.append("postpostprocessing")
    parameter_values.append(dict(interior_labels=True))
    annotation.append(f"basic cellmask segmentation: {step_n}")
    parent.append([struct_img_step, step_n-1,nuc_step])
    cellmask_step = step_n

    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("infer_cytoplasm")
    category.append("postpostprocessing")
    parameter_values.append(dict(erode_nuclei=True))
    parent.append([nuc_step, cellmask_step])
    annotation.append(f"infer cytoplasm: {step_n}")
    cyto_step = step_n


    cyto_step = step_n

    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("stack_masks")
    category.append("export")
    parameter_values.append(None)
    parent.append([nuc_step, cellmask_step, cyto_step])
    annotation.append(f"export  canonical masks (nuc,cellmask, cyto) {step_n}")
    output_step = step_n


    # ###################
    # # EXPORT
    # ###################
    # step_name.append("18")
    # function_name.append("export_PLACE_HOLDER")
    # category.append("export")
    # parameter_values.append(None)
    # parent.append([5, 17])


    ##########################
    out_dict = dict()
    for i,stepn in enumerate(step_name):

        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

-----------------
## WORKFLOW COMPOSITION HELPERS

- function to take parameters and make a workflow
- function to chain workflows


These are not nescessarily usefull


In [203]:
from typing import Union, List, Dict
def merge_workflow(workflows: List[Dict]) -> Dict:
    """
    map  parameters onto a workflow (i.e. replace params) 
    """

    def _update_steps(wf_in:Dict,wf_out, prev_last:int):
        for step, wf_step in wf_in.items():
            parent = wf_step['parent']
            new_parent=_incriment_parent(parent,prev_last)
            new_step = wf_step.copy()
            new_step['parent'] = new_parent
            new_index = str(prev_last+int(step))
            wf_out[new_index]=new_step
        return wf_out
        
    def _incriment_parent(parent: Union[int,List[int]], inc:int) -> Union[int,List[int]]:
        # assume 0 is the same for all workflows
        if isinstance(parent, list):
            new_parent = [p+inc if p>0 else p for p in parent]
        else:
            new_parent = parent+inc if parent>0 else parent
        return new_parent

    prev_last = 0
    indices = []

    for wf in workflows:
        if prev_last < 1: # simply copy the first workflow
            wf_out = wf.copy()
            prev_last =  [int(w) for w in wf_out.keys()][-1]
        else:
            wf_out = _update_steps(wf, wf_out,prev_last)
            prev_last =  [int(w) for w in wf_out.keys()][-1]
    return wf_out

## TODO: 
## make something that keeps track of "EXPORT" steps and saves a single "stack" for the end
def merge_workflow_export(workflows: List[Dict]) -> Dict:
    """
    map  parameters onto a workflow (i.e. replace params) 
    """

    def _update_steps(wf_in:Dict,wf_out, prev_last:int):
        for step, wf_step in wf_in.items():
            parent = wf_step['parent']
            new_parent=_incriment_parent(parent,prev_last)
            new_step = wf_step.copy()
            new_step['parent'] = new_parent
            new_index = str(prev_last+int(step))
            wf_out[new_index]=new_step
        return wf_out
        
    def _incriment_parent(parent: Union[int,List[int]], inc:int) -> Union[int,List[int]]:
        # assume 0 is the same for all workflows
        if isinstance(parent, list):
            new_parent = [p+inc if p>0 else p for p in parent]
        else:
            new_parent = parent+inc if parent>0 else parent
        return new_parent

    prev_last = 0
    exports = []

    for wf in workflows:
        if prev_last < 1: # simply copy the first workflow
            wf_out = wf.copy()
            prev_last =  [int(w) for w in wf_out.keys()][-1]
        else:
            wf_out = _update_steps(wf, wf_out,prev_last)
            prev_last =  [int(w) for w in wf_out.keys()][-1]

        # check if its an "export" in which case wf_out.popitem('prev_last')
        # and append exports layers
    return wf_out


In [207]:
wf1 = make_infer_masks_fromaggr_MCZ()
wf2 = make_infer_lyso_step_by_step_from_raw_dict()
wf3  = make_infer_mito_step_by_step_from_raw_dict()
wf4 = make_infer_golgi_step_by_step_from_raw_dict()
wf5 = make_infer_perox_step_by_step_from_raw_dict()
wf6 = make_infer_er_step_by_step_from_raw_dict()
wf7 = make_infer_LD_step_by_step_from_raw_dict()

# agg_wf = merge_workflow([wf1, wf2,wf3])



In [208]:
workflows = [wf1, wf2,wf3, wf4, wf5, wf6, wf7]

workflows

[{'1': {'category': 'extraction',
   'function': 'infer_nuclei_fromlabel',
   'parameter_values': {'nuc_ch': 0,
    'median_sz': 3,
    'gauss_sig': 5.0,
    'thresh_factor': 0.8,
    'thresh_min': 0.1,
    'thresh_max': 1.0,
    'max_hole_w': 35,
    'small_obj_w': 15},
   'parent': 0,
   'annotation': 'get  nuclei segmentation: 1'},
  '2': {'category': 'extraction',
   'function': 'raw_cellmask_fromaggr',
   'parameter_values': {'scale_min_max': True},
   'parent': 0,
   'annotation': ' this creates an aggregate signal for the cellmask'},
  '3': {'category': 'preprocessing',
   'function': 'scale_and_smooth',
   'parameter_values': {'median_sz': 4, 'gauss_sig': 3.4},
   'parent': 2,
   'annotation': 'cellmask segmentation: 3'},
  '4': {'category': 'preprocessing',
   'function': 'non_linear_cellmask_transform',
   'parent': 3,
   'annotation': 'cellmask segmentation: 4'},
  '5': {'category': 'core',
   'function': 'masked_object_thresh',
   'parameter_values': {'th_method': 'ave_tri_

In [209]:
merge_workflow(workflows)

{'1': {'category': 'extraction',
  'function': 'infer_nuclei_fromlabel',
  'parameter_values': {'nuc_ch': 0,
   'median_sz': 3,
   'gauss_sig': 5.0,
   'thresh_factor': 0.8,
   'thresh_min': 0.1,
   'thresh_max': 1.0,
   'max_hole_w': 35,
   'small_obj_w': 15},
  'parent': 0,
  'annotation': 'get  nuclei segmentation: 1'},
 '2': {'category': 'extraction',
  'function': 'raw_cellmask_fromaggr',
  'parameter_values': {'scale_min_max': True},
  'parent': 0,
  'annotation': ' this creates an aggregate signal for the cellmask'},
 '3': {'category': 'preprocessing',
  'function': 'scale_and_smooth',
  'parameter_values': {'median_sz': 4, 'gauss_sig': 3.4},
  'parent': 2,
  'annotation': 'cellmask segmentation: 3'},
 '4': {'category': 'preprocessing',
  'function': 'non_linear_cellmask_transform',
  'parent': 3,
  'annotation': 'cellmask segmentation: 4'},
 '5': {'category': 'core',
  'function': 'masked_object_thresh',
  'parameter_values': {'th_method': 'ave_tri_med',
   'cutoff_size': 100,


In [ ]:
wf1 = make_infer_masks_fromaggr_MCZ()
wf2 = make_infer_lyso_step_by_step_from_raw_dict()

# get a list of keys just in case things get un-ordered  
# should we use ordered dictionaries?  named tuples?



In [ ]:
indices

[1, 2, 3, 4, 5, 6, 7, 8, 9]

# DEPRICATED FOR "MASKS".json
-----------------
## NUCLEI workflow

Write the `infer_nuclei_fromlabel` spec to the widget json 

In [ ]:
from infer_subc.constants import NUC_CH


def make_infer_nuclei_dict():
    """
    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    step_n = 1
    ###################
    # EXTRACT
    ###################   
    raw_input_step = 0


    step_name.append(f"{step_n}")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = NUC_CH) )
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    step_n += 1

    ###################
    # PRE_PROCESSING
    ###################                         
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 4, gauss_sig=3.4 ))
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    step_n += 1

    ###################
    # CORE_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("apply_log_li_threshold")
    category.append("core")
    parameter_values.append(dict(thresh_factor = 0.9, 
                                                            thresh_min = .1,
                                                            thresh_max = 1.))
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    step_n += 1

    ###################
    # POST_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 25, 
                                  min_size = 15,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    step_n += 1

    
    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [ ]:

infer_nuclei_dict = make_infer_nuclei_dict()

write_workflow_json("conf_1.1.nuclei", infer_nuclei_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc/infer_subc/organelles_config/conf_1.1.nuclei.json')

--------------------------
## CYTOPLASM .json


In [ ]:
def make_infer_cytoplasm_dict():
    """
    Procedure to infer cyto from linearly unmixed input. (logical cellmask AND NOT nucleus)
    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    ###################
    # EXTRACT
    ###################   
    step_n = 1
    raw_input_step = 0


    step_name.append(f"{step_n}")
    function_name.append("infer_nuclei_fromlabel")
    category.append("extraction")
    parameter_values.append( dict(
                                nuc_ch = NUC_CH,
                                median_sz = 4,
                                gauss_sig = 1.34,
                                thresh_factor = 0.9,
                                thresh_min = 0.1,
                                thresh_max = 1.0,
                                max_hole_w = 25,
                                small_obj_w = 15)
                                )
    parent.append(raw_input_step)
    annotation.append(f"get  nuclei segmentation: {step_n}")    
    nuc_step = step_n
    step_n += 1                     

    step_name.append(f"{step_n}")
    function_name.append("infer_cellmask_fromaggr_MCZ")
    category.append("extraction")
    parameter_values.append(dict(
                                                    median_sz = 15,
                                            gauss_sig = 1.34,
                                            mo_method = "ave_tri_med",
                                            mo_adjust = 0.5,
                                            mo_cutoff_size = 150,
                                            max_hole_w = 50,
                                            small_obj_w = 45
                                            ))
    parent.append([raw_input_step, nuc_step])
    annotation.append(f" this creates an aggregate signal for the cellmask" )
    cellmask_step = step_n

    step_n += 1                     
    step_name.append(f"{step_n}")
    function_name.append("infer_cytoplasm")
    category.append("core")
    parameter_values.append(dict(erode_nuclei=True))
    parent.append([nuc_step, cellmask_step])
    annotation.append(f"basic cellmask segmentation: {step_n}")
    cyto_step = step_n


    # ###################
    # # EXPORT
    # ###################
    # step_name.append("18")
    # function_name.append("export_PLACE_HOLDER")
    # category.append("export")
    # parameter_values.append(None)
    # parent.append([5, 17])


    ##########################
    out_dict = dict()
    for i,stepn in enumerate(step_name):

        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [ ]:

infer_cytoplasm_dict = make_infer_cytoplasm_dict()

write_workflow_json("conf_1.3.cytoplasm", infer_cytoplasm_dict)